In [1]:
import pandas as pd
from database.market import Market
from transformer.date_transformer import DateTransformer
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt

In [37]:
btc = pd.read_csv("BTC-USD.csv")
eth = pd.read_csv("ETH-USD.csv")

In [42]:
for column in btc.columns:
    new_column = column.strip().replace(" ","").lower().replace("(usd)","").replace("24h","")
    btc.rename(columns={column:new_column},inplace=True)
for column in eth.columns:
    new_column = column.strip().replace(" ","_").lower().replace("(usd)","")
    eth.rename(columns={column:new_column},inplace=True)
btc.rename(columns={"adj_close":"adjclose"},inplace=True)
eth.rename(columns={"adj_close":"adjclose"},inplace=True)
btc["date"] = pd.to_datetime(btc["date"])
eth["date"] = pd.to_datetime(eth["date"])

In [43]:
eth

,date,open,high,low,close,adjclose,volume
0,2015-08-07,2.831620,3.536610,2.521120,2.772120,2.772120,1.643290e+05
1,2015-08-08,2.793760,2.798810,0.714725,0.753325,0.753325,6.741880e+05
2,2015-08-09,0.706136,0.879810,0.629191,0.701897,0.701897,5.321700e+05
3,2015-08-10,0.713989,0.729854,0.636546,0.708448,0.708448,4.052830e+05
4,2015-08-11,0.708087,1.131410,0.663235,1.067860,1.067860,1.463100e+06
...,...,...,...,...,...,...,...
2101,2021-05-08,3481.988037,3950.165039,3453.768555,3902.647705,3902.647705,5.020849e+10
2102,2021-05-09,3911.463135,3981.259033,3743.989014,3928.844727,3928.844727,5.056829e+10
2103,2021-05-10,3924.413330,4197.473145,3684.451660,3952.293945,3952.293945,6.269179e+10
2104,2021-05-11,3948.271973,4178.208984,3783.889404,4168.701172,4168.701172,5.267974e+10


In [44]:
holdings = {"btc":{},"queue":0,"bid":0}
trades = []
for exit in tqdm(range(0,20,5)):
    for floor in tqdm([True, False]):
        for signal in [1,-1]:
            for window in range(5,40,5):
                btc["rolling"] = btc["adjclose"].rolling(window=int(window)).mean()
                btc.dropna(inplace=True)
                for delta_requirement in range(0,11,1):
                    for date in btc["date"]:
                        offering = btc[btc["date"]==date]
                        offering_price = offering["adjclose"].item()
                        low = offering["low"].item()
                        high = offering["high"].item()
                        rolling = offering["rolling"]
                        if holdings["btc"] != {}:
                            if (offering_price - holdings["btc"]["buy_price"]) / holdings["btc"]["buy_price"] > delta_req uirement/100 or \
                             (exit != 0 and (date- holdings["btc"]["buy_date"]).days >= exit):
                                trade = holdings["btc"]
                                trade["sell_price"] = offering_price
                                trade["sell_date"] = date
                                trade["dr"] = delta_requirement
                                trade["window"] = window
                                trade["signal"] = signal
                                trade["floor"] = floor
                                trade["exit"] = exit
                                trades.append(trade)
                                holdings["btc"] = {}
                        if holdings["queue"] == 1 and holdings["bid"] >= low and holdings["bid"] <= high:
                            holdings["btc"] = {"buy_date":date,"buy_price":holdings["bid"]}
                            holdings["queue"] = 0 
                            holdings["bid"] = 0
                        offering_delta = (offering["rolling"].item() - offering_price) / offering_price 
                        if holdings["btc"] == {}:
                            if floor:
                                if (signal > 0 and offering_delta > 0) or (signal < 0 and offering_delta < 0):
                                    holdings["queue"] = 1
                                    holdings["bid"] = offering_price
                            else:
                                if (signal > 0 and offering_delta > delta_requirement/100) or (signal < 0 and offering_delta < -delta_requirement/100):
                                    holdings["queue"] = 1
                                    holdings["bid"] = offering_price

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [19:08<00:00, 287.01s/it]


In [45]:
ta = pd.DataFrame(trades)
analysis = []
ta["gains"] = (ta["sell_price"] - ta["buy_price"]) / ta["buy_price"]
ta["hp"] = (ta["sell_date"] - ta["buy_date"]) 
ta["hp"] = [x.days for x in ta["hp"]]
for exit in tqdm(range(0,80,10)):
    for floor in [True,False]:
        for signal in [1,-1]:
            for window in tqdm(range(10,105,5)):
                for dr in range(0,105,5):
                    dt = ta[(ta["dr"] == dr) & (ta["window"] == window) 
                            & (ta["signal"] == signal)
                             & (ta["floor"] == floor)
                            & (ta["exit"] == exit)
                           ]
                    initial = 100
                    for gains in dt["gains"]:
                        initial = initial * (1 + gains)
                    analysis.append({
                        "signal":signal,
                        "dr":dr
                        ,"window":window
                        ,"cash":initial
                        ,"floor":floor
                        ,"exit":exit
                        ,"hp":dt["hp"].mean()
                    ,"trades":len(dt)})

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 19.41it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.47it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.83it/s]

 79%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                | 15/19 [00:00<00:00, 20.40it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.43it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.14it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:01<00:00, 18.95it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 19.29it/s]

 21%|████████████████████████████████▊                                                                                                                           | 4/19 [00:00<00:00, 19.19it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 19.62it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 19.72it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.83it/s]

 74%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                        | 14/19 [00:00<00:00, 21.07it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 21.04it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 21.35it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.84it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.58it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 21.46it/s]

 16%|████████████████████████▋

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.21it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 19.64it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.93it/s]

 75%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                       | 6/8 [00:22<00:07,  3.75s/it]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.69it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.34it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.23it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 20.18it/s]



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 21.27it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 21.78it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:29<00:00,  3.75s/it]


In [46]:
a = pd.DataFrame(analysis)

In [47]:
a.sort_values("cash",ascending=False).head(20)

,signal,dr,window,cash,floor,exit,hp,trades
2,1,10,10,7287.326191,True,0,69.884615,26
23,1,10,15,4637.310152,True,0,71.916667,24
44,1,10,20,3758.460559,True,0,65.120000,25
1,1,5,10,3083.991214,True,0,54.060606,33
22,1,5,15,1703.766491,True,0,63.111111,27
43,1,5,20,1564.126959,True,0,59.925926,27
0,1,0,10,1528.933777,True,0,42.487805,41
42,1,0,20,1325.066627,True,0,32.468085,47
86,1,10,30,1195.605364,True,0,79.105263,19
65,1,10,25,1066.385559,True,0,86.444444,18
